In [7]:
from arcgis.gis import GIS
from arcgis.geometry import Point
import random
import json
import requests

import pandas as pd
import tempfile
import os

from shapely.geometry import shape, Point, Polygon
import random

import numpy as np

from getpass import getpass

In [4]:
url = "https://services5.arcgis.com/sjP4Ugu5s0dZWLjd/arcgis/rest/services/Administrative_Boundaries_Reference_(view_layer)/FeatureServer/2/query?outFields=*&where=adm0_name%3D'Afghanistan'&f=geojson"
response = requests.get(url)
afghanistan_boundaries = json.loads(response.text)

In [5]:
username = input("Enter your username: ")
password = getpass("Enter your password: ")

gis = GIS("https://www.arcgis.com", username, password)

In [11]:
# Value production per animal type
animal_values = {'F-J': 50, 'F-SA': 150, 'F-A': 120, 'M-J': 20, 'M-SA': 250, 'M-A': 5}

# Convert the GeoJSON to a shapely geometry
polygon = shape(afghanistan_boundaries['features'][0]['geometry'])


data = []
used_names = []

# List of common Afghan first names and last names
first_names = ['Ahmad', 'Ali', 'Mohammad', 'Hassan', 'Rahman', 'Khan', 'Hussein', 'Karim', 'Omar', 'Abdul']
last_names = ['Zadran', 'Niazi', 'Yousafzai', 'Ludin', 'Barakzai', 'Ahmadi', 'Kakar', 'Wardak', 'Tani', 'Noorzai']


for i in range(10000):
    while True:
        # Generate a random point within the bounding box
        lon = random.uniform(polygon.bounds[0], polygon.bounds[2])
        lat = random.uniform(polygon.bounds[1], polygon.bounds[3])
        point = Point(lon, lat)

        # Check if the point is within the polygon
        if polygon.contains(point):
            break

    # Generate random integer values for each farm animal
    animals = {animal: random.randint(0, 100) for animal in animal_values.keys()}    

    # Add the coordinates to the animals dictionary
    animals['x'] = lon
    animals['y'] = lat

    # Generate a realistic farm name and add it to the animals dictionary
    if i % 50 == 0 and i != 0 and used_names:  # Repeat a previously used name every 50th iteration
        animals['farm_name'] = random.choice(used_names)
    else:
        farm_name = f"{np.random.choice(first_names)} {np.random.choice(last_names)} Farm"
        used_names.append(farm_name)
        animals['farm_name'] = farm_name
    
    # Calculate and add the total value production estimate to the animals dictionary
    animals['total_value_production'] = sum(animals[animal] * animal_values[animal] for animal in animal_values.keys())    

    # Append the dictionary to the data list
    data.append(animals)

# Convert the data list to a DataFrame
df = pd.DataFrame(data)

NameError: name 'first_names' is not defined

In [76]:
# Create a temporary file
temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.csv')
try:
    # Write the DataFrame to the temporary file
    df.to_csv(temp_file.name, index=False)

    # Close the temporary file
    temp_file.close()

    # Create a new item from the temporary file
    item_prop = {"title": "Random Herdenmodel AFG", "type": "CSV", "tags": ["Afghanistan", "Farms"]}
    
    # Search for existing items
    existing_items = gis.content.search(query="title:'Random Herdenmodel AFG' AND owner:{}".format(gis.users.me.username), item_type="Feature Layer")
    
    # Delete existing items
    for item in existing_items:
        item.delete()

    # Add the new item
    csv_item = gis.content.add(item_properties=item_prop, data=temp_file.name)

    # Publish the CSV item as a feature layer
    feature_layer_item = csv_item.publish()

finally:
    # Delete the temporary file
    os.unlink(temp_file.name)